## **Introduction - Sentiment Analysis Part II**
So far we have tried:
- roberta pre trained
- zero shot learning
- Vader on translated tweets

Issue: none of them works well (not good classification and very imbalanced data). Potential solutions:
1. Fine-tune a model on domain-specific data: collect an Italian political sentiment dataset, fine-tune an Italian language model like AlBERTo, UmBERTo, or BERTino on this dataset. You can manually label a subset of your tweets (even ~1,000) and use active learning to scale up.
2. Use a weak supervision framework: if labeling is too expensive, use Snorkel or Skweak to create heuristic rules (e.g., hashtags, emojis, known party slogans). Combine weak labels into a probabilistic label model. Train a classifier on this noisy but large-scale data.
3. Refine your zero-shot prompt setup - If you use a zero-shot pipeline (e.g., HuggingFace pipeline("zero-shot-classification")), try:
    - Framing labels in political terms: instead of generic "positive/negative/neutral", try "supportive", "opposing", "neutral toward the politician" or their policies.
    - Use prompt engineering to improve performance: candidate_labels = ["support", "oppose", "neutral"], hypothesis_template = "This tweet expresses a {} opinion about politics."
4. Use sentiment + emotion + toxicity hybrid: often political tweets are more emotional than classically "positive" or "negative". Use an emotion classifier (anger, joy, fear, etc.). Combine with toxicity detection (like HateBERT, ToxicBERT, or Italian-specific versions).
Fuse signals into a sentiment interpretation model.

For sure, what we will do first is to consider only a filtered subset of the data, which should be the ones that are most expressive.

In [1]:
import torch
import pandas as pd
import numpy as np


## **Load the Data**

In [2]:
politicians_cleaned = pd.read_csv('politicians_data/politicians_classified.csv')

In [3]:
politicians_cleaned.head()

,Unnamed: 0,Date,ID,Content,Likes,politician,party,ideology_num,classification
0,1,2022-12-29 20:55:30+00:00,1608567407500754944,Congratulations to @netanyahu on the formation...,6021,Meloni,FdI,3,1
1,6,2022-12-27 18:00:00+00:00,1607798465198710784,Vogliamo restituire a questa Nazione l’ottimis...,14890,Meloni,FdI,3,1
2,7,2022-12-27 17:05:06+00:00,1607784649056727041,Cordiale conversazione telefonica con @Zelensk...,8589,Meloni,FdI,3,1
3,9,2022-12-25 00:08:43+00:00,1606804094102155264,Franco Frattini era un uomo garbato e intellig...,3260,Meloni,FdI,3,1
4,11,2022-12-24 07:18:01+00:00,1606549741168377856,Proteggere la libertà religiosa è un obiettivo...,5901,Meloni,FdI,3,1


In [4]:
data = politicians_cleaned.drop(columns=['Unnamed: 0', 'classification'])

## **Active Learning**
Active learning is a technique used to minimize labeling effort by selectively choosing which examples a human should label next.
How it works:
- Start with a small labeled set (e.g., 200 tweets).
- Train an initial model (using transfer learning).
- Use that model to predict on the unlabeled tweets.
- Select the most uncertain predictions (e.g., low confidence margin between classes).
- Manually label these tweets (they’re the most informative).
- Retrain the model with the new labeled data.
- Repeat 
Goal: Reach high model performance with far fewer labels than random sampling would require

SET OF LABELS:
1. Neutral / Informational: Factual updates, announcements, or descriptive statements with no clear emotional tone or stance.
2. Supportive / Affirmative: Positive endorsement or praise — for allies, institutions, values, events.
3. Attacking / Critical / Angry / Outraged: Expressing indignation, moral outrage, or rhetorical escalation. Direct attack, blame, or disapproval — usually aimed at political opponents.
4. Celebratory / Triumphant: Victory tone, pride, public accomplishments, shared wins. (molto simile a 2)
6. Call to Action / Propaganda

### **Part 1 - Label 100 random tweets**

In [34]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Load your tweets
df = data.sample(n=100, random_state=42).reset_index(drop=True)  # Use 50 for demo

# Define tone labels
tone_labels = [
    "Neutral / Informational",
    "Supportive / Affirmative",
    "Attacking / Critical",
    "Celebratory / Triumphant",
    "Angry / Outraged",
    "Call to Action / Propaganda",
    "Empathetic / Sad",
    "Alarmist / Urgent",]

# Store dropdowns and layout
dropdowns = []
for i, row in df.iterrows():
    tweet_display = widgets.HTML(value=f"<b>Tweet {i+1}</b>: {row['Content']}")
    dropdown = widgets.Dropdown(
        options=[""] + tone_labels,
        description='Tone:',
        layout=widgets.Layout(width='70%')
    )
    dropdowns.append(dropdown)
    display(tweet_display, dropdown)

# Save button logic
def save_labels(_):
    for i, dropdown in enumerate(dropdowns):
        df.at[i, 'tone_label'] = dropdown.value
    df.to_csv("labeled_tweets_with_tone.csv", index=False)
    print("✅ Labels saved to labeled_tweets_with_tone.csv")

save_button = widgets.Button(description="💾 Save Labels", button_style='success')
save_button.on_click(save_labels)
display(save_button)


HTML(value='<b>Tweet 1</b>: Alla fine di questa avventura a me interesserà una sola cosa: sapere che abbiamo f…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 2</b>: #Salvini: Ho ricordato all’avvocato Conte che mettere sullo stesso piano USA e Cin…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 3</b>: Dobbiamo spingere sulla ripartenza. Dobbiamo correre veloci, facendo investimenti …

...

HTML(value='<b>Tweet 99</b>: Busta con proiettile alla consigliera della Lega in Campania, Carmela Rescigno, i…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

HTML(value='<b>Tweet 100</b>: Partecipare alla vita pubblica è un dovere. Pericle sosteneva“un uomo che non si…

Dropdown(description='Tone:', layout=Layout(width='70%'), options=('', 'Neutral / Informational', 'Supportive …

Button(button_style='success', description='💾 Save Labels', style=ButtonStyle())

### **Part 2 - Fine Tune Model on 100 labeled tweets**

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# LOAD AND PREPROCESS LABELED DATA
df = pd.read_csv("labeled_tweets_with_tone.csv")
df = df.dropna(subset=["tone_label"])  # Remove unlabeled rows

# Label encoding
labels = df["tone_label"].unique().tolist()
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for l, i in label2id.items()}
df["label_id"] = df["tone_label"].map(label2id)

# Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Content"].tolist(), df["label_id"].tolist(), test_size=0.2, random_state=42
)


In [6]:
# TOKENIZE
from transformers import AutoTokenizer

model_name = "Musixmatch/umberto-commoncrawl-cased-v1"  # You can swap for AlBERTo if preferred
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
import torch

# CONVERT TO PYTORCH
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)


In [8]:
# FINE TUNE MODEL
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/beatricecitterio/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/bm/yxd_5px52t5d6m_z9t3mp9400000gn/T/ipykernel_2004/3046752960.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.511355
2,No log,1.449944
3,No log,1.432616


TrainOutput(global_step=30, training_loss=1.5323776245117187, metrics={'train_runtime': 20.9796, 'train_samples_per_second': 11.44, 'train_steps_per_second': 1.43, 'total_flos': 10976960005920.0, 'train_loss': 1.5323776245117187, 'epoch': 3.0})

### **Phase 3**

In [9]:
all_df = data

# Exclude already labeled ones (assuming tone_label exists only for labeled rows)
unlabeled_df = all_df[~all_df["Content"].isin(df["Content"])].reset_index(drop=True)

# Take a reasonable subset if the dataset is huge
texts_to_predict = unlabeled_df["Content"].tolist()


In [10]:
from torch.nn.functional import softmax

device = torch.device("cpu")
model.to(device)

# Make sure inputs are also on CPU
predict_encodings = tokenizer(texts_to_predict, truncation=True, padding=True, return_tensors="pt")
predict_encodings = {k: v.to(device) for k, v in predict_encodings.items()}

batch_size = 32
all_probs = []

model.eval()
with torch.no_grad():
    for i in range(0, len(texts_to_predict), batch_size):
        batch_texts = texts_to_predict[i:i+batch_size]
        batch_enc = tokenizer(batch_texts, truncation=True, padding=True, return_tensors="pt")
        batch_enc = {k: v.to("cpu") for k, v in batch_enc.items()}  # or "mps" if you fix it
        outputs = model(**batch_enc)
        batch_probs = softmax(outputs.logits, dim=1).numpy()
        all_probs.append(batch_probs)

import numpy as np
probs = np.vstack(all_probs)




KeyboardInterrupt: 

In [ ]:
import numpy as np

# Margin = difference between top two probabilities
sorted_probs = np.sort(probs, axis=1)
margins = sorted_probs[:, -1] - sorted_probs[:, -2]

# Lower margin = higher uncertainty
uncertainty_indices = np.argsort(margins)  # ascending = most uncertain first


In [ ]:
K = 50  # or any number you want to label next
uncertain_samples = unlabeled_df.iloc[uncertainty_indices[:K]].copy()

# Save for annotation
uncertain_samples.to_csv("next_batch_to_label.csv", index=False)